# Google Map Visualization

In [8]:
# Dependencies
from census import Census
# from config import (census_api_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states

# Census API Key
c = Census(census_api_key, year=2013)

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [11]:
merged_wc5_csv_for_google_df = pd.read_csv("output/merged_wc5.csv", encoding="ISO-8859-1")
# wc5_csv_for_google_df = wc5_csv_for_google_df.loc[:, ["StateDesc","CityName","PopulationCount","GeoLocation","CityFIPS"]]
# wc5_csv_for_google_df = wc5_csv_for_google_df.drop_duplicates()
merged_wc5_csv_for_google_df.count()

Unnamed: 0                    14028
Average Temperature           12712
Historic Low Temp             12712
Average High Temp             12712
Average Low Temp              12712
Historic High Temp            12712
State_x                       12712
City                          12712
CityFIPS                      14028
State_y                       14000
Place                         14000
Name                          14000
Population                    14000
Median Age                    14000
Household Income              14000
Per Capita Income             14000
Poverty Count                 14000
Poverty Rate                  14000
Unemployment Rate             14000
Year                          14028
StateAbbr                     14028
StateDesc                     14028
CityName                      14028
GeographicLevel               14028
DataSource                    14028
Category                      14028
UniqueID                      14028
Measure                     

In [12]:
google_df = []
google_df = pd.DataFrame()
google_df["lat"] = ""
google_df["long"] = ""
google_df["CityFIPS"] = ""

lat_series = []
long_series = []

for index, row in merged_wc5_csv_for_google_df.iterrows():
    latlong = str(row["GeoLocation"])
    try:
        # (33.5275663773, -86.7988174678)
        lat = latlong.split(", ")[0]
        long = latlong.split(", ")[1]
        lat = lat.replace("(","")
        long = long.replace(")","")
        google_df.loc[index, "lat"] = lat
        google_df.loc[index, "long"] = long
        google_df.loc[index, "CityFIPS"] = row["CityFIPS"]
#         print(f"{lat} and {long}")

    except Exception as e:
        print(f"{index} This skipped")
google_df.count()

lat         14028
long        14028
CityFIPS    14028
dtype: int64

In [14]:
# Get google_df to have the right number of rows
google_df = google_df.drop_duplicates(subset = ["CityFIPS"])
google_df.count()

# Get merged_wc5_csv_for_google_df to have the right number of rows
merged_wc5_csv_for_google_df = merged_wc5_csv_for_google_df.drop_duplicates(subset = ["CityFIPS"])
merged_wc5_csv_for_google_df.count()

Unnamed: 0                    500
Average Temperature           453
Historic Low Temp             453
Average High Temp             453
Average Low Temp              453
Historic High Temp            453
State_x                       453
City                          453
CityFIPS                      500
State_y                       499
Place                         499
Name                          499
Population                    499
Median Age                    499
Household Income              499
Per Capita Income             499
Poverty Count                 499
Poverty Rate                  499
Unemployment Rate             499
Year                          500
StateAbbr                     500
StateDesc                     500
CityName                      500
GeographicLevel               500
DataSource                    500
Category                      500
UniqueID                      500
Measure                       500
Data_Value_Unit               500
DataValueTypeI

In [15]:
# Store 'Lat' and 'Lng' into  locations 
# locations = census_data_complete[["Lat", "Lng"]].astype(float)
locations = google_df[["lat","long"]].astype(float)

print(locations.count())

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
poverty_rate = merged_wc5_csv_for_google_df["Poverty Rate"].astype(float)
poverty_rate = poverty_rate.drop_duplicates()
poverty_rate

lat     500
long    500
dtype: int64


0        29.326173
28        6.730769
56       16.177318
84       22.513787
112      21.866575
140      23.776511
168       7.747674
196      17.033510
224       9.089325
252       6.675538
280      20.244124
308      15.536620
336       9.831883
364      22.494451
392       9.184739
420      20.901167
448      24.149114
476      15.662976
504      23.394884
532      24.530624
560      23.092297
588      18.332889
616      23.450812
644      13.800881
672      15.905917
700      14.740232
728      20.047288
756      20.244079
784      17.336336
812      16.982946
           ...    
13188    29.665577
13216    31.483592
13244    15.373709
13272    19.318356
13300     6.131274
13328    16.792512
13356    12.445023
13384    19.218719
13412    16.633541
13440    21.863071
13468    19.518131
13496    28.091920
13524    20.854497
13552    11.873514
13580    15.456541
13608    22.381830
13636    22.263467
13664    12.739351
13692    18.592973
13720    18.787213
13748     8.024982
13776    22.

In [19]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))